In [ ]:
%load_ext autoreload
%autoreload 2
%run -n main.py

# datasets

In [ ]:
# path = join_path(DATA_DIR, DATASET)
# !mkdir -p {path}

In [ ]:
# for name in [NE5, BSNLP, FACTRU, GAREEV]:
#     path = join_path(CORUS_DATA_DIR, CORUS_FILES[name])
#     records = LOADS[name](path)
#     records = (_.adapted for _ in records)
#     records = log_progress(records, desc=name)

#     path = join_path(DATA_DIR, DATASET, name + JL + GZ)
#     items = as_jsons(records)
#     lines = format_jl(items)
#     dump_gz_lines(lines, path)

In [ ]:
%run -n main.py
datasets = {}
for name in [NE5, BSNLP, FACTRU, GAREEV]:
    path = join_path(DATA_DIR, DATASET, name + JL + GZ)
    lines = load_gz_lines(path)
    items = parse_jl(lines)
    datasets[name] = list(from_jsons(items, Markup))

# models

In [ ]:
# for name in [MITIE, NATASHA, PULLENTI, TEXTERRA, TOMITA, DEEPPAVLOV, DEEPPAVLOV_BERT, SLOVNET_BERT]:
#     path = join_path(DATA_DIR, name)
#     !mkdir -p {path}

## cpu

In [ ]:
# docker = docker_client()

In [ ]:
# for model_name in [MITIE, TOMITA, NATASHA, PULLENTI, TEXTERRA]:
#     log(model_name)
#     model = MODELS[model_name]()
#     model.start(docker)
#     model.wait()

#     for dataset_name in [NE5, BSNLP, FACTRU, GAREEV]:
#         records = model.map(_.text for _ in datasets[dataset_name])
#         records = (_.adapted for _ in records)
#         records = log_progress(records, desc=dataset_name)

#         path = join_path(DATA_DIR, model_name, dataset_name + JL + GZ)
#         items = as_jsons(records)
#         lines = format_jl(items)
#         dump_gz_lines(lines, path)
    
#     model.stop(docker)

## gpu

In [ ]:
# # select cuda 10.0, high bandwith
# !vast search offers | grep '1 x  GTX 1080 Ti'

In [ ]:
# %run -n main
# # model = DeeppavlovModel()
# # model = DeeppavlovBERTModel()
# model = SlovnetBERTModel()

In [ ]:
# !vast create instance 498744 --image {model.image} --disk 20

In [ ]:
# !vast show instances

In [ ]:
# !ssh ssh4.vast.ai -p 20837 -l root -Nf -L {model.port}:localhost:{model.container_port}

In [ ]:
# for dataset_name in [NE5, BSNLP, FACTRU, GAREEV]:
#     records = datasets[dataset_name]
#     records = log_progress(records, desc=dataset_name)
#     records = model.map(_.text for _ in records)
#     records = (_.adapted for _ in records)

#     path = join_path(DATA_DIR, model.name, dataset_name + JL + GZ)
#     items = as_jsons(records)
#     lines = format_jl(items)
#     dump_gz_lines(lines, path)

In [ ]:
# !vast destroy instance 500837

# score

In [ ]:
dataset_models = {}
for dataset in DATASETS:
    for model in MODELS:
        path = join_path(DATA_DIR, model, dataset + JL + GZ)
        lines = load_gz_lines(path)
        items = parse_jl(lines)
        dataset_models[dataset, model] = list(from_jsons(items, Markup))

In [ ]:
scores = {}
for dataset, model in log_progress(dataset_models):
    preds = dataset_models[dataset, model]
    targets = datasets[dataset]
    scores[dataset, model] = score_markups(preds, targets)

# report

In [ ]:
table = scores_report_table(scores, DATASETS, MODELS)
html = format_scores_report(table)
HTML(html)

In [ ]:
html = format_github_scores_report(table)
patch_readme(NER1, html, README)
patch_readme(NER1, html, SLOVNET_README)
HTML(html)

In [ ]:
BENCH = [
    # GTX 1080 Ti
    Bench(
        DEEPPAVLOV,
        init=5.9,
        disk=1 * GB,  # 1GB emb + 5MB model
        ram=3 * GB,
        speed=24.31,  # 1.95 / 7 cores on CPU,
        device=GPU
    ),
    Bench(
        DEEPPAVLOV_BERT,
        init=34.5,
        disk=2 * GB,
        ram=6 * GB,
        speed=13.13,  # 17.71 / 3 cores on CPU
        device=GPU
    ),
    Bench(
        SLOVNET_BERT,
        init=5,
        disk=473 * MB,
        ram=1959 * MB,
        speed=17.5,
        device=GPU
    ),
    
    # 16 CPUs
    Bench(
        PULLENTI,
        init=2.85,
        disk=16 * MB,
        ram=253 * MB,
        speed=6.05
    ),
    Bench(
        TEXTERRA,
        init=47.6,
        disk=193 * MB,
        ram=3.3 * GB,  # leaks
        speed=20.16 / 5  # utils ~5 cores
    ),
    Bench(
        TOMITA,
        init=2.03,
        disk=64 * MB,
        ram=63 * MB,
        speed=29.8,
    ),
    Bench(
        NATASHA,
        init=2.05,
        disk=700 * KB,
        ram=160 * MB,
        speed=8.8,
    ),
    Bench(
        MITIE,
        init=28.3,
        disk=327 * MB,
        ram=261 * MB,
        speed=32.8,
    ),    
]

html = format_bench_report(BENCH, MODELS)
patch_readme(NER2, html, README)
patch_readme(NER2, html, SLOVNET_README)
HTML(html)